In [1]:
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels
import hvplot.xarray

In [2]:
station = "Calcofi"

## Load DATA


In [3]:
data_obs = xr.open_dataset(f"../data/1_products/{station}_obs_zoo_climato_monthly_2002_2015.zarr", engine="zarr").load()
data_obs

<xarray.Dataset> Size: 102kB
Dimensions:    (time: 156, latitude: 9, longitude: 9, layer: 1)
Coordinates:
  * latitude   (latitude) float64 72B 29.5 30.5 31.5 32.5 ... 35.5 36.5 37.5
  * layer      (layer) int64 8B 0
  * longitude  (longitude) float64 72B -125.5 -124.5 -123.5 ... -118.5 -117.5
  * time       (time) datetime64[ns] 1kB 2002-01-31 2002-02-28 ... 2014-12-31
Data variables:
    day        (time, latitude, longitude, layer) float32 51kB nan nan ... nan
    night      (time, latitude, longitude, layer) float32 51kB nan nan ... nan
Attributes:
    desc:     Aggregated and smoothed observations using lowess filtre.

In [4]:
data_cmems = xr.open_dataset(f"../data/1_products/{station}_cmems.zarr", engine="zarr").load()
data_cmems = data_cmems.resample(time="1D").mean(dim="time").interpolate_na(dim="time", method="linear")
first_year = str(data_obs.indexes["time"].to_pydatetime()[0].year)
last_year = str(data_obs.indexes["time"].to_pydatetime()[-1].year)
data_cmems = data_cmems.sel(time=slice(first_year, last_year)).sel(
    latitude=data_obs.latitude, longitude=data_obs.longitude
)
data_cmems

<xarray.Dataset> Size: 25MB
Dimensions:              (depth: 3, latitude: 9, longitude: 9, time: 4748)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 72B 29.5 30.5 31.5 ... 36.5 37.5
  * longitude            (longitude) float64 72B -125.5 -124.5 ... -118.5 -117.5
  * time                 (time) datetime64[ns] 38kB 2002-01-01 ... 2014-12-31
Data variables:
    T                    (time, depth, latitude, longitude) float64 9MB 17.23...
    npp                  (time, latitude, longitude) float64 3MB 318.0 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 9MB 115.7...
    zooc                 (time, latitude, longitude) float64 3MB 0.6602 ... nan
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Climato


In [5]:
for i in range(1, 13):
    print(
        f"Month {i} : ",
        set(
            pd.DataFrame({"month": data_cmems.indexes["time"].month, "day": data_cmems.indexes["time"].day})
            .query(f"month == {i}")["day"]
            .unique()
        ),
    )

Month 1 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 2 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29}
Month 3 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 4 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 5 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 6 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 7 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 8 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}


In [6]:
data_cmems_climato = data_cmems.groupby("time.dayofyear").mean()
data_cmems_climato

<xarray.Dataset> Size: 2MB
Dimensions:              (dayofyear: 366, depth: 3, latitude: 9, longitude: 9)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 72B 29.5 30.5 31.5 ... 36.5 37.5
  * longitude            (longitude) float64 72B -125.5 -124.5 ... -118.5 -117.5
  * dayofyear            (dayofyear) int64 3kB 1 2 3 4 5 ... 362 363 364 365 366
Data variables:
    T                    (dayofyear, depth, latitude, longitude) float64 712kB ...
    npp                  (dayofyear, latitude, longitude) float64 237kB 361.2...
    pelagic_layer_depth  (dayofyear, depth, latitude, longitude) float64 712kB ...
    zooc                 (dayofyear, latitude, longitude) float64 237kB 0.816...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Expand to original time series interval


In [7]:
all_years = []
for year in range(int(data_cmems.time.min().dt.year), int(data_cmems.time.max().dt.year + 1)):
    slice_max = 366
    if not pd.Timestamp(f"{year}-12-31").is_leap_year:
        slice_max = 365

    yearly_data = data_cmems_climato.sel(dayofyear=slice(1, slice_max)).assign_coords(
        time=(
            "dayofyear",
            pd.date_range(
                start=f"{year}-01-01",
                end=f"{year}-12-31",
                freq="D",
                name="time",
            ),
        ),
    )

    all_years.append(yearly_data.swap_dims({"dayofyear": "time"}))
data_cmems_climato_expanded = xr.concat(all_years, dim="time").drop_vars("dayofyear")
data_cmems_climato_expanded.time.attrs = data_cmems.time.attrs
data_cmems_climato_expanded

<xarray.Dataset> Size: 25MB
Dimensions:              (time: 4748, depth: 3, latitude: 9, longitude: 9)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 72B 29.5 30.5 31.5 ... 36.5 37.5
  * longitude            (longitude) float64 72B -125.5 -124.5 ... -118.5 -117.5
  * time                 (time) datetime64[ns] 38kB 2002-01-01 ... 2014-12-31
Data variables:
    T                    (time, depth, latitude, longitude) float64 9MB 16.72...
    npp                  (time, latitude, longitude) float64 3MB 361.2 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 9MB 110.1...
    zooc                 (time, latitude, longitude) float64 3MB 0.8161 ... nan
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Plot


In [8]:
data_cmems_climato_expanded.mean(["latitude", "longitude"]).hvplot.line(
    x="time",
    groupby="depth",
    title=f"Climato {station} - CMEMS T",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'2f2e4120-454f-4b05-9bcd-f0605a8ae45d': {'version…

## Export


In [9]:
data_cmems_climato_expanded.to_zarr(
    f"../data/1_products/{station}_cmems_climato.zarr",
    mode="w",
)